# Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [2]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
KEYWORDS = ['python', 'парсинг','SSD','Марс'] 

In [2]:
res = requests.get('https://habr.com/ru/all/')
soup_ = BeautifulSoup(res.text, 'lxml')


In [39]:
#собираю список ссылок со страницы
news = soup_.find_all('a', class_ = "post__title_link")

href_list = []
for new in news:
    href_ = new.get('href')
    href_list.append(href_)
    
#href_list


In [40]:
# Функция проверки текста на наличие ключей
def check_paper (KEYWORDS,text):
    for key in KEYWORDS:
        if key in str(text):        
            return True
      
#check_paper (KEYWORDS,text) 

In [48]:
#Создаю и наполняю датафрейм 
frame = pd.DataFrame(columns=['Заголовок', 'Дата', 'Ссылка', 'Текст статьи'])

for link in href_list:
    soup_ = BeautifulSoup(requests.get(link).text, 'lxml')
    text = soup_.find('div', class_ = 'post__body post__body_full').text
    check = check_paper (KEYWORDS,text) 
    if check == True:
        heder = soup_.title.text
        date = soup_.find('span', class_ = 'post__time').get('data-time_published')
        df2 = pd.DataFrame({'Заголовок' : [heder], 'Дата' : [date], 'Ссылка' : [link], 'Текст статьи' : [text]})
        frame = pd.concat([frame, df2])
   
    time.sleep(0.3)

frame  

,Заголовок,Дата,Ссылка,Текст статьи
0,Учёные раскрыли универсальную геометрию геолог...,2020-12-18T10:35Z,https://habr.com/ru/post/533752/,\nУпражнения в чистой математике привели к соз...
0,Энергия и энтропия / Хабр,2020-12-18T10:18Z,https://habr.com/ru/post/533740/,\nЭта статья является конспектом материала пос...
0,Марс — наш / Блог компании VDSina.ru хостинг с...,2020-12-18T09:45Z,https://habr.com/ru/company/vdsina/blog/532430/,\n\n\r\nМарс — потенциально пригодная для жизн...


# Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [3]:
EMAIL = ['89066390161@yandex.ru','maksuxa4@yandex.ru','maks@yandex.ru']
import json

In [4]:
Headers = {   
'Accept': "application/json, text/plain, */*",
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Connection': 'keep-alive',
'Content-Length': '32',
'Content-Type': 'application/json;charset=UTF-8',
'Host': 'identityprotection.avast.com',
'Origin': 'https://www.avast.com',
'Referer': 'https://www.avast.com/',
#'sec-ch-ua': "Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87",
'sec-ch-ua-mobile': '?0',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

In [5]:
#Тут так и не понял, где искать post запрос.

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data = {'emailAddresses': EMAIL}
#Headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
req = requests.post(URL, data = json.dumps(data), headers = Headers)

req

<Response [200]>

In [56]:
type(req.text)
decoded = json.loads(req.text)

decoded

{'breaches': {'16868': {'breachId': 16868,
   'site': 'borealis.su',
   'recordsCount': 349061,
   'description': "At an unconfirmed date, Russian Minecraft server Borealis.su's database was allegedly breached. The stolen data contains usernames, passwords, and email addresses. This breach is being privately shared on the internet.",
   'publishDate': '2019-08-08T00:00:00Z',
   'statistics': {'usernames': 349054, 'passwords': 349039, 'emails': 345545}},
  '4466': {'breachId': 4466,
   'site': 'vipdiablo.com',
   'recordsCount': 481153,
   'description': "In July 2016, VIP Diablo's user database was allegedly breached. The stolen data contains email addresses and passwords. The compromised data was made public in March 2017, on a few hacker forum and sites.",
   'publishDate': '2017-05-26T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 477980, 'emails': 481153}},
  '16768': {'breachId': 16768,
   'site': 'canva.com',
   'recordsCount': 137434010,
   'description': "In May 201

In [57]:
site_breaches = pd.DataFrame(req.json()['breaches'])
#columns=['Email', 'date','site','description'])
site_breaches = site_breaches.transpose()
site_breaches

,breachId,site,recordsCount,description,publishDate,statistics
16868,16868,borealis.su,349061,"At an unconfirmed date, Russian Minecraft serv...",2019-08-08T00:00:00Z,"{'usernames': 349054, 'passwords': 349039, 'em..."
4466,4466,vipdiablo.com,481153,"In July 2016, VIP Diablo's user database was a...",2017-05-26T00:00:00Z,"{'usernames': 0, 'passwords': 477980, 'emails'..."
16768,16768,canva.com,137434010,"In May 2019, graphic-design site Canva's datab...",2019-06-13T00:00:00Z,"{'usernames': 137304104, 'passwords': 61348974..."
22306,22306,egrul-piter.ru,193684,"In November 2020, a collection of over 23,000 ...",2020-12-03T00:00:00Z,"{'usernames': 0, 'passwords': 192290, 'emails'..."
29000,29000,job.ws,221434,"In November 2020, a collection of over 23,000 ...",2020-12-03T00:00:00Z,"{'usernames': 0, 'passwords': 218013, 'emails'..."
29233,29233,meetmaster.ru,397347,"In November 2020, a collection of over 23,000 ...",2020-12-10T00:00:00Z,"{'usernames': 0, 'passwords': 397326, 'emails'..."
16935,16935,xrumer.winguild.ru,277201,"At an unconfirmed date, Russian audio equipmen...",2019-09-12T00:00:00Z,"{'usernames': 0, 'passwords': 275497, 'emails'..."
16866,16866,firstvds.ru,125140,"At an unconfirmed date, Russian server hosting...",2019-08-08T00:00:00Z,"{'usernames': 121480, 'passwords': 125051, 'em..."
18150,18150,gonitro.com,86440975,"In October 2020, the PDF editor Nitro was alle...",2020-11-19T00:00:00Z,"{'usernames': 0, 'passwords': 18103956, 'email..."
17457,17457,vkmix.com,2127865,"At an unconfirmed date, the Russian social med...",2020-04-09T00:00:00Z,"{'usernames': 0, 'passwords': 2127865, 'emails..."


In [125]:
# В таблице отсутствуют e-mail. Их можно вытащить из другого датафрейма mail_breaches

mail_breaches = pd.DataFrame(req.json()['summary'])
dfValues = list(data_breaches['breaches'])
print(dfValues)
list_email = list(data_breaches['breaches'].index)
print(list_email)
mail_breaches

[[], [4466, 16768, 4102], [16868, 22306, 29000, 29233, 16935, 16866, 18150, 17457, 33945, 18122, 16659]]
['89066390161@yandex.ru', 'maksuxa4@yandex.ru', 'maks@yandex.ru']


,89066390161@yandex.ru,maksuxa4@yandex.ru,maks@yandex.ru
breaches,[],"[4466, 16768, 4102]","[16868, 22306, 29000, 29233, 16935, 16866, 181..."


In [115]:
#Напишу функцию для проверки наличия breaches id в списке
#  dfValues, чтобы понимать к какому мейлу относиться утечка

def proverka_breachId (par):
    index = 0
    for Val in dfValues:        
        if par in Val:        
            return list_email[index]
        index += 1
        
print(proverka_breachId(16659))

maks@yandex.ru


In [124]:
#Применяю функцию и удаляю лишние столбцы
site_breaches['Email'] = site_breaches['breachId'].apply(proverka_breachId)
#del site_breaches['breachId'],['recordsCount'],['statistics']
site_breaches.drop(['breachId', 'recordsCount','statistics'], axis = 1)

,site,description,publishDate,Email
16868,borealis.su,"At an unconfirmed date, Russian Minecraft serv...",2019-08-08T00:00:00Z,maks@yandex.ru
4466,vipdiablo.com,"In July 2016, VIP Diablo's user database was a...",2017-05-26T00:00:00Z,maksuxa4@yandex.ru
16768,canva.com,"In May 2019, graphic-design site Canva's datab...",2019-06-13T00:00:00Z,maksuxa4@yandex.ru
22306,egrul-piter.ru,"In November 2020, a collection of over 23,000 ...",2020-12-03T00:00:00Z,maks@yandex.ru
29000,job.ws,"In November 2020, a collection of over 23,000 ...",2020-12-03T00:00:00Z,maks@yandex.ru
29233,meetmaster.ru,"In November 2020, a collection of over 23,000 ...",2020-12-10T00:00:00Z,maks@yandex.ru
16935,xrumer.winguild.ru,"At an unconfirmed date, Russian audio equipmen...",2019-09-12T00:00:00Z,maks@yandex.ru
16866,firstvds.ru,"At an unconfirmed date, Russian server hosting...",2019-08-08T00:00:00Z,maks@yandex.ru
18150,gonitro.com,"In October 2020, the PDF editor Nitro was alle...",2020-11-19T00:00:00Z,maks@yandex.ru
17457,vkmix.com,"At an unconfirmed date, the Russian social med...",2020-04-09T00:00:00Z,maks@yandex.ru


Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [56]:
f = open ('/Users/maksim/Netology/material/token.txt')
token = f.read()
print(token)

b94f71df2d135dced40605fe20535d066d711c9121c75ec76cae2f97db53608148c99f296029ca98b6b42


In [57]:
wall_get = 'https://api.vk.com/method/wall.get?'
version = '5.84'
GROUP = 'netology' 
SLEEP = 0.33

In [58]:
params =  {
    'access_token': token,
    'v': version,
    'count': 10,
    'domain': GROUP
}

In [62]:
res = requests.get(wall_get, params)
res.json()

{'response': {'count': 7423,
  'items': [{'id': 72756,
    'from_id': -30159897,
    'owner_id': -30159897,
    'date': 1609149901,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Наш предновогодний digital-марафон завершён! \n \nСпасибо, что провели этот месяц с нами ❤ Надеемся, вы зарядились новогодним настроением и верой в чудеса. \n \nПобедителями последней недели стали: \n— [id124744560|Аделина Каримуллина] \n— [id159217528|Сания Шёгрен] \n— [id44357824|Дарья Сафарова], [id142769953|Элеонора Захваткина] и [id4528290|Tonja Volkova] разделили третье место \n \nА самыми активными участниками нашего сообщество по итогам 4-х недель становятся: \n1 место — [id119945220|Артур Каримуллин] \n2 место — [id427483117|Tatyana Proshina] \n3 место — [id192002422|Андрей Егоров] \n \nПоздравляем! \nЧтобы получить свой приз — напишите нам в сообщения сообщества.',
    'is_pinned': 1,
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -7,
       'date': 1609148851,
     

In [63]:
from datetime import datetime
data = pd.DataFrame(res.json()['response']['items'], columns=['date', 'text'])
data
#ts = data['date'][1]
#print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M'))

,date,text
0,1609149901,Наш предновогодний digital-марафон завершён! \...
1,1609250760,Сегодня вышел последний выпуск ЭтоИзи в уходящ...
2,1609226580,Нам свойственна тяга к саморазвитию: мы стреми...
3,1609167360,Сегодня отмечается международный день кино 🎬 \...
4,1609141560,📆 19 января стартует бесплатный курс «Soft Ski...
5,1609067520,"«Если хотите стать писателем, вы должны, прежд..."
6,1608981960,Каждый месяц пользователи со всего мира просма...
7,1608966360,*партнёрский пост* \n \nУже сегодня в 12:00 за...
8,1608908220,Так ли умён искусственный интеллект? 🤔 \n \nИс...
9,1608881160,Давайте уже после праздников! \n \nПодборка бе...


In [61]:
def date_convert (par):
    return datetime.utcfromtimestamp(par).strftime('%Y-%m-%d %H:%M:%S')
    
data['date_'] = data['date'].apply(date_convert)

del data['date'] 
data


,text,date_
0,Наш предновогодний digital-марафон завершён! \...,2020-12-28 10:05:01
1,Сегодня вышел последний выпуск ЭтоИзи в уходящ...,2020-12-29 14:06:00
2,Нам свойственна тяга к саморазвитию: мы стреми...,2020-12-29 07:23:00
3,Сегодня отмечается международный день кино 🎬 \...,2020-12-28 14:56:00
4,📆 19 января стартует бесплатный курс «Soft Ski...,2020-12-28 07:46:00
5,"«Если хотите стать писателем, вы должны, прежд...",2020-12-27 11:12:00
6,Каждый месяц пользователи со всего мира просма...,2020-12-26 11:26:00
7,*партнёрский пост* \n \nУже сегодня в 12:00 за...,2020-12-26 07:06:00
8,Так ли умён искусственный интеллект? 🤔 \n \nИс...,2020-12-25 14:57:00
9,Давайте уже после праздников! \n \nПодборка бе...,2020-12-25 07:26:00
